In [85]:
from sklearn.datasets import load_breast_cancer

# 데이터는 파이썬의 자료구조인 딕셔너리 형태입니다.
cancer = load_breast_cancer()

# 딕셔너리 데이터는 key, value로 이루어져있습니다.
# 키를 이용하면 데이터의 값에 접근할 수 있기 때문에 먼저 key를 출력해봅니다.
cancer.keys()

dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])

In [86]:
import pandas as pd

# 독립변수를 가져옵니다.
data_list = cancer["data"]
# 독립변수의 이름을 가져옵니다.
column_list = cancer["feature_names"]
# 종속변수를 가져옵니다. (유방암 여부)
target_list = cancer["target"]

# 먼저, 독립변수만을 가지고 데이터프레임을 만들어줍니다.
data = pd.DataFrame(data=data_list, columns=column_list)
# feature 이름을 target으로 지정하고 종속변수를 넣어줍니다.
data["target"] = target_list

# 데이터의 1. 행의 개수, 2) 열의 개수 출력
print(data.shape)

# 생성한 데이터 프레임 확인
data.head(1)

(569, 31)


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189,0


In [87]:
# 랜덤 포레스트 모델을 가져옵니다.
# 앙상블 기법에 속합니다.
from sklearn.ensemble import RandomForestClassifier

# 랜덤 포레스트 모델을 새성합니다.
rf = RandomForestClassifier()

# 독립변수를 만들어줍니다.
# target은 종속변수이기 때문에 제거해줍니다.
x = data.drop(columns=["target"], axis=1)

# 종속변수를 만들어줍니다.
y = data["target"]

In [94]:
# 평가를 위한 cross_validate를 가져오고, 교차검증을 위한 KFold를 가져옵니다.
from sklearn.model_selection import cross_validate, KFold, cross_val_predict

# 평가지표 5개를 가져옵니다. 
# 정확도, 정밀도, 재현율, f1_score, auc
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# 교차검증을 설정합니다.
# n_splits=20의 의미는 전체의 19/20은 훈련, 1/20은 테스트 데이터로 사용한다는 의미입니다.
# 이렇게 하면 총 20번 교차검증을 수행합니다.
k_fold = KFold(n_splits=20, random_state=1, shuffle=True)

# 평가지표를 객체로 만들어줍니다.
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score),
           'roc_auc_score': make_scorer(roc_auc_score)
          }

result = cross_val_predict(rf, x, y, cv=k_fold, method="predict_proba")

# 평가 결과가 가진 키를 출력해봅니다.
result

/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/skyepodium/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/s

array([[0.9, 0.1],
       [0.8, 0.2],
       [1. , 0. ],
       ...,
       [1. , 0. ],
       [1. , 0. ],
       [0. , 1. ]])

In [99]:
print(len(x))

# for item in result:
#     print(item)

569


In [89]:
# 20번 교차검증을 수행했기 때문에 20개의 평가지표들의 평균값을 구해줍니다.
accuracy = result["test_accuracy"].mean()
precision = result["test_precision"].mean()
recall = result["test_recall"].mean()
f1_score = result["test_f1_score"].mean()
auc_score = result["test_roc_auc_score"].mean()

print("accuracy: {0: .4f}".format(accuracy))
print("precision: {0: .4f}".format(precision))
print("recall: {0: .4f}".format(recall))
print("f1_score: {0: .4f}".format(f1_score))
print("auc_score: {0: .4f}".format(auc_score))

accuracy:  0.9562
precision:  0.9609
recall:  0.9721
f1_score:  0.9651
auc_score:  0.9504
